In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
# licenses

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# Connect to client libraries

openai = OpenAI()

ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
models = ["gpt-5-nano", 'llama3.2:latest', "Llama 3.1 8B"]
clients = {"gpt-5": openai, "llama3.2:latest": ollama, "Llama 3.1 8B": ollama}

dummy_job_desc = """
Job Description: Senior Fullstack Engineer (FinTech)
Role: Senior Fullstack Engineer Location: Remote / Hybrid (Madrid) Stack Required:

Java: Minimum 8 years of experience. Expert knowledge of Spring Boot and Microservices architecture.

Angular: Minimum 5 years of experience. Deep understanding of RxJS, NgRx, and State Management.

Cloud: Experience with AWS (EC2, S3, Lambda).

Databases: Proficiency in PostgreSQL and Redis.

Soft Skills: Leadership of small teams and experience in Agile/Scrum environments.

Testing: Unit testing with JUnit and Jasmine/Karma.

CI/CD: Experience with Jenkins and Docker.

Languages: Native/C1 English level."""

dummy_cv = """
Alejandro Martínez
Professional Summary Senior Developer with over 9 years of experience in the software industry, specialized in building scalable enterprise applications.

Work Experience

Lead Developer @ TechFlow Solutions | Jan 2020 – Present (6 years)

Leading a team of 4 developers using Agile methodologies.

Architecting microservices using Java 17 and Spring Boot.

Developed complex dashboards using Angular 14, RxJS, and NgRx.

Deployment and orchestration using Docker and Jenkins pipelines.

Managing databases with PostgreSQL.

Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019 (3.5 years)

Fullstack development using Java (Spring) and Angular 4.

Worked extensively with JUnit for backend testing.

Managed caching strategies using Redis.

Collaborated in a Scrum environment.

Junior Developer @ StartupGen | June 2014 – May 2016 (2 years)

Backend development with Java.

Maintenance of legacy SQL databases.

Skills & Tools

Languages: Java (Expert), TypeScript, SQL.

Frameworks: Spring Boot, Angular, Express.js.

DevOps: Docker, Jenkins, Git.

Languages: English (C1 - Advanced), Spanish (Native).
"""


In [ ]:
def fnCompare(cv, job_desc, selected_model):
    client = clients[selected_model]
    response = client.chat.completions.create(
        model=selected_model,
        messages=[
            {"role": "system", "content": """
             You are an expert Technical Recruiter and Data Analyst.
             Your task is to analyze a Job Description and a Candidate CV to identify skill matches.
             For each skill mentioned in the Job Description, you must:
             Identify if the skill exists in the CV.
             Calculate the total years of experience for that specific skill based on the employment dates provided.
             List the skills from the Job Description that are MISSING in the CV.
             Be precise with dates.
             If a skill is mentioned in a role, assume the duration of that role applies to the skill unless otherwise specified.
             """},
            {"role": "user", "content": f"CV: {cv}\nJob description: {job_desc}"}
        ]
    )
    return response.choices[0].message.content

In [ ]:
# gradio view

with gr.Blocks(title="CV matcher") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
           cv = gr.TextArea(label="CV", value=dummy_cv)
        with gr.Column(scale=6):
            job_desc = gr.TextArea(label="Job description", value=dummy_job_desc)

    with gr.Row():
        with gr.Column(scale=6):
            selected_model = gr.Dropdown(label="Select a Model", choices=models, value=models[1], show_label=False)
        with gr.Column(scale=6):
            btnCompare = gr.Button("Compare")

    with gr.Row():
        output = gr.TextArea(label="Output")

    
    btnCompare.click(fn=fnCompare, inputs=[cv, job_desc, selected_model], outputs=[output])

ui.launch()

In [ ]:
# results:

"""gpt-5-nano (model frontier)
Here is a skill-by-skill match analysis based on the Job Description (Senior Fullstack Engineer, FinTech) and Alejandro Martínez’s CV.

Job Description skills, CV presence, and calculated experience

- Java
  - CV exists: Yes
  - Experience (total from roles): 11.5 years
    - Jun 2014 – May 2016: Java backend (2 years)
    - Jun 2016 – Dec 2019: Java/Spring (3.5 years)
    - Jan 2020 – Present: Java 17 (6 years)
  - Evidence in CV: “Backend development with Java” (StartupGen), “Fullstack development using Java (Spring)” (GlobalBank), “Architecting microservices using Java 17” (Lead Developer)

- Spring Boot
  - CV exists: Yes
  - Experience: 6 years
    - Jan 2020 – Present: Spring Boot as part of microservices architecture
  - Evidence: Lead Developer role states “Architecting microservices using Java 17 and Spring Boot”

- Microservices architecture
  - CV exists: Yes
  - Experience: 6 years
    - Jan 2020 – Present (Lead Developer): Architecting microservices with Spring Boot
  - Evidence: Lead Developer, “Architecting microservices”

- Angular
  - CV exists: Yes
  - Experience: 9.5 years
    - 2016–2019: Angular 4 (3.5 years)
    - 2020–Present: Angular 14 (6 years)
  - Evidence: Lead Developer role with Angular 14; 2016–2019 role with Angular 4

- RxJS
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: Angular usage in Lead Developer role (RxJS mentioned)

- NgRx
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: Angular with NgRx in the Lead Developer role

- State Management
  - CV exists: Yes
  - Experience: 6 years (via NgRx)
  - Evidence: NgRx usage in Angular in the Lead Developer role

- AWS (EC2, S3, Lambda)
  - CV exists: No
  - Experience: 0 years
  - Evidence: No AWS shown in CV

- PostgreSQL
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: “Managing databases with PostgreSQL” in Lead Developer role

- Redis
  - CV exists: Yes
  - Experience: 3.5 years
  - Evidence: “Managed caching strategies using Redis” in GlobalBank Systems

- Soft skills: Leadership of small teams
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: Leading a team of 4 developers in Lead Developer role

- Agile/Scrum environments
  - CV exists: Yes
  - Experience: 9.5 years
  - Evidence: Lead Developer (Agile methodologies); GlobalBank Systems (Scrum environment)

- JUnit
  - CV exists: Yes
  - Experience: 3.5 years
  - Evidence: GlobalBank Systems role states “JUnit for backend testing”

- Jasmine/Karma (testing)
  - CV exists: No
  - Experience: 0 years
  - Evidence: No mention of Jasmine or Karma in CV

- Jenkins
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: “Jenkins pipelines” in Lead Developer role

- Docker
  - CV exists: Yes
  - Experience: 6 years
  - Evidence: “Deployment and orchestration using Docker” in Lead Developer role

- English level (Native/C1)
  - CV exists: Yes
  - Experience: Not time-bound (language proficiency, not duration)
  - Evidence: English (C1 - Advanced) listed in CV

Skills from the Job Description that are MISSING in the CV
- AWS (Cloud) – no evidence of AWS experience
- Jasmine/Karma – no evidence of Jasmine or Karma testing
Notes: All other major required areas (Java, Spring Boot, Microservices, Angular, RxJS, NgRx, State Management, PostgreSQL, Redis, JUnit, Jenkins, Docker, Agile/Scrum, Leadership) are represented in the CV with corresponding durations.

Summary of fit
- Strong matches: Java, Spring Boot, Microservices, Angular (with RxJS/NgRx/State management), PostgreSQL, Redis, Jenkins, Docker, Agile/Scrum, JUnit, Leadership.
- Gaps: AWS cloud experience is not shown; Jasmine/Karma testing not shown.
- English: Meets C1 requirement.

If you’d like, I can format this as a candidate scorecard (weights per requirement) or generate targeted interview questions to probe the missing AWS and Jasmine/Karma areas.

-----
llama3.2:latest

Let's analyze the job description and candidate CV.

**Matching Skills:**

Java:
- Skill exists in CV.
- Employment dates for Java: 
  - Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019 (3.5 years).
  - Lead Developer @ TechFlow Solutions | Jan 2020 – Present (so far, 3+ years).

Total experience with Java: approximately 6.5 years.

 TypeScript:
- Skill does not exist in CV.



Angular:
- Skill exists in CV.
- Employment dates for Angular:
  - Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019 (for Angular 4, around 2.5+ years).
  - Lead Developer @ TechFlow Solutions | Jan 2020 – Present (for Angular 14, approximately 3+ years).

Total experience with Angular: approximately 5.5 years.



PostgreSQL:
- Skill exists in CV.
- Employment dates for PostgreSQL:
  - Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019 (1 year with management of legacy SQL databases).
  - Lead Developer @ TechFlow Solutions | Jan 2020 – Present.

Total experience with PostgreSQL: approximately 3.5 years.



AWS (EC2, S3, Lambda):
- Skill does not exist in CV.



Redis:
- Skill exists in CV.
- Employment dates for Redis:
  - Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019.

Total experience with Redis: approximately 3 years.



Spring Boot:
- Skill exists in CV.
- Employment dates for Spring Boot:
  - Lead Developer @ TechFlow Solutions | Jan 2020 – Present (for using in the context of microservices architecture).

Total experience with Spring Boot: approximately 2+ years.



Jenkins:
- Skill does exist in CV, but the description only mentions Docker and Jenkins pipelines.
- Employment dates for Jenkins are not explicitly mentioned.

Total related experience with Jenkins: approximately 3.5 years.



Docker:
- Employment dates for Docker:
  - Lead Developer @ TechFlow Solutions | Jan 2020 – Present (for deployment and orchestration).
  
Total experience with Docker: approximately 2+ years.



DevOps tools:
- Skill exists in CV, but the description is incomplete.
- We cannot accurately calculate the total related experience without knowing more.



Unit testing with JUnit:
- Skill does exist in CV.
- Employment dates for unit testing (JUnit): same as Software Engineer @ GlobalBank Systems | June 2016 – Dec 2019.

Total experience with unit testing using JUnit: approximately 3.5 years.



Jasmine/Karma or Unit Testing with Jasmine/Karma:
- Skill exists, but with Angular expertise. No clear employment dates are mentioned.

CI/CD:
- Skill does not exist fully in the CV.
However some part of CI/CD (orchestration with Jenkins) is mentioned.


Language Skills: 
English and Spanish
For this example we'll count experience with one language at a time.


Total English Experience that counts would be considered from Junior Software Engineer to Lead Developer roles as there's no explicit mention of continuous learning or speaking practice.
For the full stack development job skills with English are "minimum" but required by more emphasis.



**Missing skills:**

- AWS (EC2, S3, Lambda)
- TypeScript
- Cloud Architecture



"""